In [0]:
%sql
CREATE CATALOG IF NOT EXISTS DEV
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstorage7cvu443bw3t6e.dfs.core.windows.net/145196962537236';

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS QA
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstorage7cvu443bw3t6e.dfs.core.windows.net/145196962537236';

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS DEV.demo_db;
CREATE DATABASE IF NOT EXISTS QA.demo_db;

In [0]:
%sql
DROP DATABASE IF EXISTS DEV.default CASCADE;

In [0]:
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

flight_time_df = (spark.read.format("json")
                    .schema(flight_schema_ddl)
                    .option("dateFormat", "M/d/y")
                    .load("abfss://dbfs-container@shaiklove.dfs.core.windows.net/External/CH7-Data Files/dataset_ch7/flight-time.json")
)

In [0]:
display(flight_time_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS DEV.demo_db.flight_time_tbl (
    FL_DATE DATE, 
    OP_CARRIER STRING, 
    OP_CARRIER_FL_NUM INT, 
    ORIGIN STRING, 
    ORIGIN_CITY_NAME STRING, 
    DEST STRING, 
    DEST_CITY_NAME STRING, 
    CRS_DEP_TIME INT, 
    DEP_TIME INT, 
    WHEELS_ON INT, 
    TAXI_IN INT, 
    CRS_ARR_TIME INT, 
    ARR_TIME INT, 
    CANCELLED STRING, 
    DISTANCE INT
) USING DELTA


#LOAD DATA INTO DELTA TABLE

In [0]:
flight_time_df.write.format("delta").mode("append").saveAsTable("DEV.demo_db.flight_time_tbl")

In [0]:
flight_time_tbl = spark.read.table("DEV.demo_db.flight_time_tbl")
display(flight_time_tbl)

**CREATE A DELTA TABLE USING DELTATABLE API**

In [0]:
# Read and display the JSON data
df = (spark.read
    .format("json")
    .load("abfss://dbfs-container@shaiklove.dfs.core.windows.net/External/CH7-Data Files/dataset_ch7/flight-time.json")
)
display(df)

# Save the DataFrame as a Delta table
df.write.format("delta").save("abfss://dbfs-container@shaiklove.dfs.core.windows.net/External/CH7-Data Files/dataset_ch7/flight-time-delta")

from delta import DeltaTable

# Create or replace the Delta table
(DeltaTable.createOrReplace(spark)
    .tableName("DEV.demo_db.flight_time_tbl")
    .addColumn("id", "INT")
    .addColumn("FL_DATE", "DATE")
    .addColumn("OP_CARRIER", "STRING")
    .addColumn("OP_CARRIER_FL_NUM", "INT")
    .addColumn("ORIGIN", "STRING")
    .addColumn("ORIGIN_CITY_NAME", "STRING")
    .addColumn("DEST", "STRING") 
    .addColumn("DEST_CITY_NAME", "STRING")
    .addColumn("CRS_DEP_TIME", "INT")
    .addColumn("DEP_TIME", "INT")
    .addColumn("WHEELS_ON", "INT")
    .addColumn("TAXI_IN", "INT")
    .addColumn("CRS_ARR_TIME", "INT")
    .addColumn("ARR_TIME", "INT")
    .addColumn("CANCELLED", "STRING")
    .addColumn("DISTANCE", "INT")
    .execute()
)

In [0]:
%sql
select * from DEV.demo_db.flight_time_tbl

In [0]:
SAVE DELTA DATA IN EXTERNAL LOCATION

In [0]:
(flight_time_df.coalesce(1)
    .write
    .format("parquet")
    .mode("overwrite")
    .save("abfss://dbfs-container@shaiklove.dfs.core.windows.net/External/CH7-Data Files/dataset_ch7")
)